#PLAN LEAF RECOGNITION

####Package Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import seaborn as sns
from zipfile import ZipFile
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
import numpy as np
import math
from termcolor import colored, cprint

###Download Datasets

####Flavia Dataset

In [ ]:
!gdown "https://drive.google.com/uc?id=1wN6c6ZL-ulMQK3ePq-8RgK_8EqoJsm5h"

In [ ]:
!unzip leaves.zip
!rm *.zip

####Swedish Dataset

In [ ]:
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf1.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf2.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf3.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf4.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf5.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf6.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf7.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf8.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf9.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf10.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf11.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf12.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf13.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf14.zip
!wget http://www.cvl.isy.liu.se/en/research/datasets/swedish-leaf/leaf15.zip

In [ ]:
def extract(zip):
  zipper = ZipFile(zip,'r')
  zipper.extractall('./' + zip.split('.')[0])

In [ ]:
_ = [extract(zip) for zip in os.listdir('.') if zip.endswith(".zip")]

In [ ]:
!rm *.zip

#### LeafSnap Dataset

In [ ]:
!wget http://leafsnap.com/static/dataset/leafsnap-dataset.tar
!tar xf leafsnap-dataset.tar && rm leafsnap-dataset.tar leafsnap-dataset-readme.txt

## Unify  Datasets

In [ ]:
!mkdir Dataset
!mv leaf* Dataset/
!mv dataset/images/field/* Dataset/
!rm /content/Dataset/leafsnap-dataset-images.txt
!mv Leaves_Named/* /content/Dataset/

In [ ]:
!rm -rf Leaves_Named/ dataset/ sample_data/

In [ ]:
path = os.getcwd() + '/Dataset'
for root, dirs, files in os.walk(path, topdown=False):
  for dir in dirs:
    if dir[0].isupper() & (dir.lower() in dirs):
      src_imgs = os.listdir(os.path.join(path,dir))
      for img in src_imgs:
        os.rename(os.path.join(path,dir,img),os.path.join(path,dir.lower(),img))
      os.rmdir(os.path.join(path,dir))
    elif(dir[0].isupper()):
      os.rename(os.path.join(path,dir),os.path.join(path,dir.lower()))

## Preprocessing

#### Convert non .jpg files to .jpg format

In [ ]:
path = os.getcwd() + '/Dataset'
for root, dirs, files in os.walk(path, topdown=False):
  for name in files:
    if os.path.splitext(os.path.join(root, name))[1].lower() != ".jpg":
      outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
      im = Image.open(os.path.join(root, name))
      cprint("Generating jpeg for " + name, 'green', attrs=['bold'])
      im.thumbnail(im.size)
      im.save(outfile, "JPEG", quality=100)
      !rm {os.path.join(root, name)}

#### Resizing to 512 x 512

In [ ]:
path = os.getcwd() + '/Dataset'
for root, dirs, files in os.walk(path, topdown=False):
  for name in files:
    if os.path.splitext(os.path.join(root, name))[1].lower() == ".jpg":
      im = Image.open(os.path.join(root, name))
      if im.size != (512,512):
        cprint("Resizing " + os.path.join(root, name) + " from " + str(im.size) + " to (512,512)", 'green', attrs=['bold'])
        im1 = im.resize((512,512))
        im1.save(os.path.join(root, name))
      else:
        cprint("The size of " + os.path.join(root, name) + "is (512,512)!!", 'green', attrs=['bold'])

#### Importing images

In [ ]:
targetSize = (512,512)
dir2dataset = "Dataset"

In [ ]:
trainAug = ImageDataGenerator(
  #samplewise_center=True,
	rotation_range=25,
	fill_mode="nearest",
  rescale=1./255,
  zoom_range=0.3,
  horizontal_flip=True,
  vertical_flip=True,
  validation_split = 0.2
  #samplewise_std_normalization=True,
  )
traindata = trainAug.flow_from_directory(
    dir2dataset,
    target_size=targetSize,
    color_mode="rgb",
    class_mode="categorical",
    batch_size=12,
    shuffle=True,
    seed=1337,
    subset = 'training'
)
testdata = trainAug.flow_from_directory(
    dir2dataset,
    target_size=targetSize,
    color_mode="rgb",
    class_mode="categorical",
    batch_size=12,
    shuffle=False,
    seed=1337,
    subset = 'validation'
)
print("\n")
cprint("Total Number of Images Found: "+ str(traindata.samples + testdata.samples), 'green', attrs=['bold'])
print("\n")
cprint("Mapping of plant species to integer ->", 'green', attrs=['bold'])
i = 0
for key, value in traindata.class_indices.items():
  i = i + 1
  if (i < 5) | (i > 221):
    cprint ("{:<30} : {:<10}".format(key, value), 'cyan', attrs=['bold'])

# cprint(str(traindata.class_indices), 'red', attrs=['bold'])

In [ ]:
batchX, batchy = traindata.next()
cprint('Batch shape=%s, %s' % (batchX.shape, batchy.shape), 'blue')

## Apply Deep Learning Algorithm



### Training the Model

#### Setting variables

In [ ]:
input_shape = (512,512,3)
numClasses = traindata.num_classes

#### Model - Xception

In [ ]:
model2 = tf.keras.applications.Xception(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=225,
    classifier_activation="softmax",
)


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 1e-2)
model2.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/MyDrive/8th sem/Technical seminar/weights/tripletake49_{epoch}",
        monitor="loss",
        verbose=1,
    )
]


In [ ]:
model2 = tf.keras.models.load_model("/content/drive/MyDrive/8th sem/Technical seminar/weights/tripletake59_1")

In [ ]:
H= model2.fit(traindata,epochs=60,callbacks=callbacks)